<a href="https://colab.research.google.com/github/ekingit/hackathon/blob/main/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/hackathon/plot.py /content/plot.py

from plot import create_plots

Mounted at /content/drive


In [7]:
!pip install tsfresh
from tsfresh import extract_features
import pandas as pd


def get_features(df, label):
    df = df.astype("float32")
    df_features = extract_features(
        df.reset_index().melt(id_vars = 'timestamp', var_name = 'id', value_name = 'value'),
            column_id = 'id',
            column_sort = 'timestamp',
            column_value = 'value',
            n_jobs = 1,  # Use single process to avoid progress bar issues
            disable_progressbar = False,
            default_fc_parameters = {
                "absolute_sum_of_changes": None,
                "count_above_mean": None,
                "has_duplicate": None,
                "has_duplicate_max": None,
                "has_duplicate_min": None,
                "longest_strike_above_mean": None,
                "mean_abs_change": None,
                "percentage_of_reoccurring_values_to_all_values": None,
                "root_mean_square": None,
                "mean": None,
                "median": None,
                "skewness": None,
                "sum_values": None,
                "variance": None,
                "benford_correlation": None,
                "cid_ce": [{"normalize": True}],
                "fft_aggregated": [{"aggtype": "centroid"}],

            }
    )
    df_features['label'] = label
    return df_features


def create_df_features(df_real, df_synth):
    df_real_features = get_features(df_real, 1)
    df_synth_features = get_features(df_synth, 0)
    df_features = pd.concat([df_real_features, df_synth_features])
    df_features = df_features.loc[:, df_features.isna().sum() == 0]
    return df_features

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.5/176.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.2 which is incompatible.


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
real_path = '/content/drive/MyDrive/hackathon/smart_meters_london_2013.csv'
synth_path = '/content/drive/MyDrive/hackathon/synthetic_data.csv'

df_real = pd.read_csv(real_path, parse_dates = ['timestamp']).set_index('timestamp')
df_synth = pd.read_csv(synth_path, parse_dates = ['timestamp']).set_index('timestamp')

In [4]:
fig_dict, rmse_dict = create_plots(df_real, df_synth)

In [5]:
score = 0
for item in rmse_dict.values():
    score += item.loc[item['statistic'] != 'median', 'value'].sum()

print(score)

17.309769203290607


In [8]:
df_features = create_df_features(df_real, df_synth)

X = df_features.drop('label', axis = 1).astype(float)
y = df_features['label'].astype(int)

Feature Extraction: 100%|██████████| 4126/4126 [01:52<00:00, 36.59it/s]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
y_train = np.array(y_train)
y_test = np.array(y_test)

model = xgb.XGBClassifier(eval_metric = 'logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy:.4f}')

Test accuracy: 0.9994


In [10]:
featureImportance = model.feature_importances_
df_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': featureImportance
})
df_importance = df_importance.sort_values(by= 'Importance', ascending = False)
df_importance

,Feature,Importance
7,value__percentage_of_reoccurring_values_to_all...,0.993071
1,value__count_above_mean,0.003387
13,value__variance,0.002063
14,value__cid_ce__normalize_True,0.000870
0,value__absolute_sum_of_changes,0.000219
4,value__has_duplicate_min,0.000166
10,value__median,0.000126
8,value__root_mean_square,0.000039
11,value__skewness,0.000032
5,value__longest_strike_above_mean,0.000026
